In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [ ]:
#每个批次的大小
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

#初始化权值
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1)) #生成一个截断的正态分布

#初始化偏置
def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

#卷积层
def conv2d(x,W):
    #x input tensor of shape `[batch, in_height, in_width, in_channels]` 形状是tensor；黑白图片通道(channels)数是1，彩色是3
    #W  a filter / kernel tensor of shape`[filter_height, filter_width, in_channels, out_channels]`
    #W是一个滤波器或卷积核，形状是tensor
    #`strides[0] = strides[3] = 1` strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`. same padding会在外层补 0 保持一致
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    # ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') #采用最大池化的方式，kszie代表窗口的大小

#定义三个placeholder
x = tf.placeholder(tf.float32,[None,784]) #28 * 28转换一维向量
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

#改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

#把x_image和权值向量进行卷积，再加上偏度值，然后应用于relu激活函数
# 28*28*1 的图片卷积之后变为28*28*32 使用same padding大小不变
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化之后变为 14*14*32
h_pool1 = max_pool_2x2(h_conv1) #进行max_pooling

#初始化第二个卷积层的权值和偏置
# 第二次卷积之后变为 14*14*64
W_conv2 = weight_variable([5,5,32,64]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_vairable([64]) 每个卷积核一个偏置值
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
# 第二次池化之后变为 7*7*64
h_pool2 = max_pool_2x2(h_conv2) #进行max_pooling


# 初始化第一个全连接层
W_fc1 = weight_variable([7*7*64,1024]) #上一层有7*7*64个神经元，全连接层有1024个神经元
b_fc1 = bias_vairable([1024])
# 7*7*64的图像变成1维向量，把第二个池化层的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64]) #-1代表真实的批次数100
#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
#keep_prob用来表示神经元的输出概率
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) #一部分神经元工作，一部分不工作

# 初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_vairable([10])
#-------------------*******************--------------------
# logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
# prediction = tf.nn.sigmoid(logits)

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
# train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# prediction_2 = tf.nn.softmax(prediction)
# correct_prediction = (tf.equal(tf.argmax(prediction_2,1), tf.argmax(y,1)))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#-------------------*******************--------------------

#计算输出
logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
prediction = tf.nn.softmax( tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

#交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔列表中
correct_prediction = (tf.equal(tf.argmax(prediction_2,1), tf.argmax(y,1))) #argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
            
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter: " + str(epoch) + ", acc: " + str(acc))